In [48]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
from activation_functions import sigmoid
from metrics import accuracy
from BaseRegression import BaseRegression
from collections import Counter

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Decision Tree

Decision tree is a powerful algorithm that can fit complex data and perform both classification, regression, and multioutput tasks.

Advantage of Decision Trees:
* Make very few assumptions about the data.
* Fairly intuitive and the decisions are easy to interpret. (white box model)
* Feature scaling and centering is not necessary to obtain good results.

Other noteworthy info:
* Decision trees form the fundamental components of a RandomForest.
* The CART algorithm (scikit-learn) produces only binary trees whereas ID3 for example allows nodes to have more than 2 children.

<img src="https://miro.medium.com/v2/resize:fit:1060/1*H6thrs5CR_wdxQyMCwWawQ.png" alt="Image of Decision tree" style="background-color:white;">

### <span style="color:#217AB8"> Making predictions</span> 

Starting at the root node and follow the conditions that apply to your current instance to the leaf. This will look like is the attribute x of your instance larger or smaller than 1. If yes follow the tree down the right path. If no go down the left. Once you reach a leaf node (aka does not have any child nodes) use this node's class to predict the class of your instance.\
It takes approximately $O(log_2(m))$ nodes to predict an instance's class. This is indipendent of the number of features so predictions are very fast even with large training sets.

* *sample* how many training samples a node's condition applies to.\
* *value* how many training samples of each class a node applies to (eg. in node x: class a is represented 1, class b is represented 20 times)
* *gini* measures the impurity of a node (pure when a node applies to only instances of one class)

Gini impurity = $G_i = 1  - \sum_{k=1}^n p_{i,k}^2$ where $p_{i,k}$ is the ratio of class k instances among the training instances in the ith node.

### <span style="color:#217AB8">CART classification and regression tree algorithm</span> 
A greedy algorithm meaning at every step from the beginning it tries to optimize the split rather than checking whether this improves impurity further down the line at lower levels.\
Therefore it does not guarantee an optimal solution.\
It is also an NP complete problem and requires O(exp(m)) time, making it hard to work with even small training sets. -> find reasonalbly good solutions.

1. Split the training set into 2 using a single feature $k$ and a threshold $t_k$ (eg. petal length >= 1.3). Find the purest subsets for pairs (k, $t_k$) weighted by their size.

&emsp;&emsp;&emsp;Minimize Cost function: $ J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$ where m is the number of instances in the subesets

2. Continue this on the subsets recursively.
3. Stop when max_depth is reached or if no split further reduces the gini impurity.\
Training complexity requires the comparison of all features (unless max_features is set) on all samples at each node.\
This brings the training compexity to $O(n * m * log(m))$

If the tree is left unconstrained it will fit itself very closely to the training data and most likely overfitting.\
This is often described as a non-parametric model. In contrast, parametric models such as linear models have a pre-determined number of parameters, so their degrees of freedom are limited.\
(which in turn can lead to underfitting, especially when the data contains more complex patterns than the model is able to catch)



### <span style="color:#217AB8">Gini impurity or Entropy</span> 
Shannon's information theory: Entropy measures the average information content of a message:\
Entropy is 0 when all messages are identical.\
For example if in the case of decision tree the entropy is 0 then the node captures only one class.

Entropy in the ith node: $H_i = \sum_{k=1}^{n} p_{i,k}*log(p_{i,k})$ where $p_{i,k}\not=0$

[Article explaining entropy, information gain and gini](https://www.machinelearningnuggets.com/splitting-criteria-in-decision-trees/#:~:text=Entropy%20measures%20data%20points%27%20degree,ranges%20between%200%20and%201.&text=We%20can%20see%20that%20the,the%20data%20is%20perfectly%20randomized.
)

Example:

Should you use Gini or Entropy?
According to O'reilly "Hands on Machine learning" they lead to similar trees.\
"The gini index seems to be slightly faster to compute so it is a good default.\
However when they differ the Gini impurity tends to isolate the most frequent class  in its own branch of the tree, while entropy tends to produce slightly more balanced trees." (Sebastian Raschka's analysis)

### <span style="color:#217AB8">Regularization</span> 
For example:
* restrict the maximum depth of the tree
* set the minumum number of samples a node must have before it can split
* set the minumum number of samples a leaf must have
* set the minumum fraction of all training data that a leaf must represent
* restrict the maximum number of leaf nodes that can be determined
* restrict the maximum number of features considered for splitting at a node

You could also train the tree without restrictions and then prune the tree after the training.
For example:
Prune a node if all of its children are leaves and it provides no statistically significant improvement of purity.\
Using the chi-sqaure test with null hypothesis that the node increases purity. If you want to reject the null hypothesis with 95% confidence then the p-value should be over 0.05. 
 


In [49]:

def entropy(classes):
    # Entropy measures data points' degree of impurity, uncertainty, or surprise. 
    # Range [0 and 1]: equals 1 when the data is perfectly randomized.
    # Expected Value of surprise
    probs = np.bincount(classes) / len(classes)
    return -np.sum([px * np.log2(px) for px in probs if px > 0])

def info_gain(labels, column):
    # column: is the chosen attribute column of the dataset
    # labels: the target array y
    # The information gain of an attribute
    # I(attribute a) = Entropy of dataset - sum over unique values v in a{( count(v)/len(y) * Entropy(Unique value in a))}
    dataset_entropy = entropy(labels)
    sum_val_entropies = 0
    for val in np.unique(column):
        value_ids = [id for id, elem in enumerate(column) if elem == val]
        val_clss=[clss for id, clss in enumerate(labels) if id in value_ids]
        sum_val_entropies += column.count(val) / len(column) * entropy(val_clss)
    return dataset_entropy - sum_val_entropies

def gini(classes):
    #compute the gini impurity
    probs = np.bincount(classes) / len(classes)
    return 1 - np.sum(np.square(probs))


In [50]:
clss = [1,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,4,4,4,4,4,4,4,4,4]
vls = ['b','b','b','b','b','b','c','c','c','b','b','b','b','a','a','a','b','b','b','b','a','a','a','a','a']

info_gain(clss, vls)

0.6186951736180384

In [51]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right
        self.gini = None
        self.samples = None
        self.clss = None
    
    def is_leaf_node(self):
        return self.value is not None

In [52]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split =min_samples_split
        self.max_depth = max_depth
        self.n_features =n_features 
        self.root = None

    def fit(self, X, y):
        # ensure that the number of features to use is never larger than the actual existing features.
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        # grow the tree with its choices and thresholds
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check for stopping criteria before continuing to grow the tree.
        if (depth == self.max_depth 
            or n_samples < self.min_samples_split
            or n_labels == 1):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # if stopping criteria is not met countinue growing the tree.
        feature_ids = np.random.choice(n_feats, self.n_features, replace=False)
        # greedy search for best features and thresholds
        best_feat, best_thresh = self._best_criteria(X,y, feature_ids)
        left_ids, right_ids = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_ids, :], y[left_ids], depth+1)
        right = self._grow_tree(X[right_ids, :], y[right_ids], depth+1)
        return Node(best_feat, best_thresh, left, right)


    def _info_gain(self, labels, column, threshold):
        # column: is the chosen attribute column of the dataset
        # labels: the target array y
        # The information gain of an attribute
        # I(attribute a) = Entropy of dataset - sum over unique values v in a{( count(v)/len(y) * Entropy(Unique value in a))}
        parent_entropy = entropy(labels)
        left_ids, right_ids = self._split(column, threshold)
        
        if len(left_ids) == 0 or len(right_ids) == 0:
            return 0
        
        left_entropy, right_entropy = entropy(labels[left_ids]), entropy(labels[right_ids])
        child_entropy = (len(left_ids) / len(labels)) * left_entropy + (len(right_ids) / len(labels)) * right_entropy
        return parent_entropy - child_entropy

    def _split(self, col, split_thresh):
        left_ids = np.argwhere(col >= split_thresh).flatten()
        right_ids = np.argwhere(col < split_thresh).flatten()
        return left_ids, right_ids

    def _most_common_label(self, y):
        most_common = Counter(y).most_common(1)[0][0]
        return most_common

    def _best_criteria(self, X, y, feat_ids):
        best_gain = -1
        split_id, split_thresh = None, None
        for feat_id in feat_ids:
            X_col = X[:, feat_id]
            threshold = np.unique(X_col)
            for thresh in threshold:
                gain = self._info_gain(y, X_col, thresh)
                if gain > best_gain:
                    best_gain = gain
                    split_id = feat_id
                    split_thresh = thresh
        return split_id, split_thresh


    def predict(self, X):
        # for each sample run through the tree and predict the class of its leaf node.
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        # recursively so check for stopping condition first
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


In [53]:
cancer_data = datasets.load_breast_cancer()
X = cancer_data.data
y = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [54]:
tree = DecisionTree(max_depth=15)
tree.fit(X_train, y_train)

predicted = tree.predict(X_test)
accuracy = accuracy(predicted, y_test)
print(accuracy)

0.09649122807017543
